In [1]:
import model_lib
import resnet_pretrain
import tensorflow as tf
import os

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = model_lib.Model()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
checkpoint = tf.train.Checkpoint(model=model, global_step=tf.Variable(0, dtype=tf.int64))
checkpoint.restore('checkpoint/ckpt-99').expect_partial()
global_step = checkpoint.global_step

In [4]:
global_step.numpy()

2513100

In [5]:
def save(model, global_step):
    #Export as SavedModel for finetuning and inference.
    saved_model = build_saved_model(model)
    export_dir = os.path.join('checkpoint', 'saved_model')
    checkpoint_export_dir = os.path.join(export_dir, str(global_step.numpy()))
    if tf.io.gfile.exists(checkpoint_export_dir):
        tf.io.gfile.rmtree(checkpoint_export_dir)
    tf.saved_model.save(saved_model, checkpoint_export_dir)

    #if FLAGS.keep_hub_module_max > 0:
        # Delete old exported SavedModels.
    exported_steps = []
    for subdir in tf.io.gfile.listdir(export_dir):
        if not subdir.isdigit():
            continue
        exported_steps.append(int(subdir))
    exported_steps.sort()
    for step_to_delete in exported_steps[:-1]:
        tf.io.gfile.rmtree(os.path.join(export_dir, str(step_to_delete)))


def build_saved_model(model):
    #Returns a tf.Module for saving to SavedModel.
    class SimCLRModel(tf.Module):
       #Saved model for exporting to hub.

        def __init__(self, model):
            self.model = model
            # This can't be called `trainable_variables` because `tf.Module` has a getter with the same name.
            self.trainable_variables_list = model.trainable_variables
        
        @tf.function
        def __call__(self, inputs, trainable):
            self.model(inputs, training=trainable)
            return get_salient_tensors_dict()

    module = SimCLRModel(model)
    input_spec = tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32)
    module.__call__.get_concrete_function(input_spec, trainable=True)
    module.__call__.get_concrete_function(input_spec, trainable=False)
    return module


def get_salient_tensors_dict():
    #Returns a dictionary of tensors.
    graph = tf.compat.v1.get_default_graph()
    result = {}
    for i in range(1, 5):
        result['block_group%d' % i] = graph.get_tensor_by_name('resnet/block_group%d/block_group%d:0' % (i, i))
    
    result['initial_conv'] = graph.get_tensor_by_name('resnet/initial_conv/Identity:0')
    result['initial_max_pool'] = graph.get_tensor_by_name('resnet/initial_max_pool/Identity:0')
    result['final_avg_pool'] = graph.get_tensor_by_name('resnet/final_avg_pool:0')
    result['logits'] = graph.get_tensor_by_name('proj_h/proj_head_output:0')

    return result

In [6]:
save(model, global_step)

INFO:tensorflow:Assets written to: checkpoint/saved_model/2513100/assets


INFO:tensorflow:Assets written to: checkpoint/saved_model/2513100/assets
